<a href="https://colab.research.google.com/github/Suffragette/github-slideshow/blob/master/3_labels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install pyspark==3.1.2
!pip install spark-nlp==3.3.1

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

import findspark
findspark.init()

from pyspark.sql import SparkSession
import sparknlp

spark = SparkSession.builder \
    .appName("Spark NLP") \
    .master("local[4]") \
    .config("spark.driver.memory","16G") \
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:3.3.1") \
    .getOrCreate()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)


Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 336 kB in 1s (237 kB/s)
Reading package lists... Done
Spark NLP version:  3.3.1
Apache Spark version:  3.1.2


In [56]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [57]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.getOrCreate()

# Load the data
# Load the data
labels = spark.read.csv("/content/drive/MyDrive/Chatgpt/Τεχνικές Διαχείρισης και Εξόρυξης για  Δεδομένα Μεγάλου Όγκου/Medical-Abstracts-TC-Corpus-main/medical_tc_labels.csv", inferSchema=True, header=True)
train = spark.read.csv("/content/drive/MyDrive/Chatgpt/Τεχνικές Διαχείρισης και Εξόρυξης για  Δεδομένα Μεγάλου Όγκου/Medical-Abstracts-TC-Corpus-main/medical_tc_train.csv", inferSchema=True, header=True)
test = spark.read.csv("/content/drive/MyDrive/Chatgpt/Τεχνικές Διαχείρισης και Εξόρυξης για  Δεδομένα Μεγάλου Όγκου/Medical-Abstracts-TC-Corpus-main/medical_tc_test.csv", inferSchema=True, header=True)


# Show the first few rows of the DataFrame
data.show()


+---------------+--------------------+
|condition_label|    medical_abstract|
+---------------+--------------------+
|              5|Tissue changes ar...|
|              1|Neuropeptide Y an...|
|              2|Sexually transmit...|
|              1|Lipolytic factors...|
|              3|Does carotid rest...|
|              3|The shoulder in m...|
|              2|The management of...|
|              4|Pharmacomechanica...|
|              5|Color Doppler dia...|
|              5|Noninvasive diagn...|
|              4|Sodium sensitive ...|
|              1|Imaging bone tumo...|
|              5|Closure of a bron...|
|              1|Utility of frozen...|
|              4|Antihypertensive ...|
|              2|Gallbladder perfo...|
|              1|Left ventricular ...|
|              4|Tongue ischemia f...|
|              5|In vitro and in v...|
|              1|Chondrosarcoma of...|
+---------------+--------------------+
only showing top 20 rows



In [58]:
print(data.columns)


['condition_label', 'medical_abstract']


In [62]:
from sparknlp.base import DocumentAssembler, Finisher
from sparknlp.annotator import Tokenizer, Normalizer, StopWordsCleaner

# Assemble the text column into a Document type column
document_assembler = DocumentAssembler() \
    .setInputCol("medical_abstract") \
    .setOutputCol("document")

# Tokenize the document
tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

# Normalize the tokens (remove punctuation and convert to lowercase)
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")

# Remove stopwords
stopwords_cleaner = StopWordsCleaner() \
    .setInputCols(["normalized"]) \
    .setOutputCol("clean_tokens") \
    .setCaseSensitive(False)

# Finish the preprocessing (convert the result back to a string column)
finisher = Finisher() \
    .setInputCols(["clean_tokens"]) \
    .setOutputCols(["clean_text"]) \
    .setOutputAsArray(False) \
    .setCleanAnnotations(False)

# Define the stages of the pipeline
stages = [document_assembler, tokenizer, normalizer, stopwords_cleaner, finisher]

# Create a pipeline
from pyspark.ml import Pipeline
pipeline = Pipeline().setStages(stages)

# Fit the pipeline to the training data
model = pipeline.fit(train)

# Transform the training and test data
train_clean = model.transform(train)
test_clean = model.transform(test)

# Fit the pipeline to the test data and transform it
test_clean = model.transform(test)

# Show the first few rows of the cleaned training data
train_clean.show()

# Show the first few rows of the cleaned test data
test_clean.show()



+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|condition_label|    medical_abstract|            document|               token|          normalized|        clean_tokens|          clean_text|
+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|              5|Tissue changes ar...|[{document, 0, 10...|[{token, 0, 5, Ti...|[{token, 0, 5, Ti...|[{token, 0, 5, Ti...|Tissue@changes@ar...|
|              1|Neuropeptide Y an...|[{document, 0, 12...|[{token, 0, 11, N...|[{token, 0, 11, N...|[{token, 0, 11, N...|Neuropeptide@Y@ne...|
|              2|Sexually transmit...|[{document, 0, 17...|[{token, 0, 7, Se...|[{token, 0, 7, Se...|[{token, 0, 7, Se...|Sexually@transmit...|
|              1|Lipolytic factors...|[{document, 0, 10...|[{token, 0, 8, Li...|[{token, 0, 8, Li...|[{token, 0, 8, Li...|Lipolytic@fact